<a href="https://colab.research.google.com/github/LokeshVadlamudi/graded3DeepLearning/blob/master/graded3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.vision import *

In [3]:
import cv2
from tqdm import tqdm
import numpy as np
import pandas as pd

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import zipfile

from keras.preprocessing import image
import urllib.request
from keras.preprocessing.image import *
import keras
from keras.layers import *

Using TensorFlow backend.


In [4]:
df_train = pd.read_csv('/content/drive/My Drive/planetDataset/train_v2.csv')
df_train.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=15yUQp_EY3FTWajcTLP7HiZsnfI7WhZEm'
fluff, id = link.split('=')


downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train-jpg.zip')


with zipfile.ZipFile('train-jpg.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [0]:
# !unzip '/content/drive/My Drive/planetDataset/train-jpg.zip'

In [0]:
x_train = []
x_test = []
y_train = []

In [0]:
def flatten(l):
    result = []
    for sublist in l:     
        for item in sublist: 
            result.append(item)
    return result

In [0]:
labs = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

In [0]:
lab_m = {l: i for i, l in enumerate(labs)}
inv_lab_m = {i: l for l, i in lab_m.items()}


In [13]:
for f, tags in tqdm(df_train.values, miniters=5000):
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[lab_m[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

100%|██████████| 40479/40479 [00:55<00:00, 732.58it/s]


In [0]:
splitter = 30000
x_train, x_valid, y_train, y_valid = x_train[:splitter], x_train[splitter:], y_train[:splitter], y_train[splitter:]

In [15]:
print(x_train.shape, x_valid.shape, y_train.shape, y_valid.shape)

(30000, 32, 32, 3) (10479, 32, 32, 3) (30000, 17) (10479, 17)


In [0]:
def loki_model():
  loki = keras.models.Sequential()
  loki.add(Conv2D(64, (3, 3), input_shape=(32, 32, 3),activation='relu'))
  loki.add(MaxPooling2D(pool_size=(2, 2)))
  loki.add(Conv2D(32, (3, 3), activation='relu'))
  loki.add(MaxPooling2D(pool_size=(2, 2)))
  loki.add(Flatten())
  loki.add(Dense(units=128, activation='relu'))
  loki.add(Dropout(0.5))
  loki.add(Dense(units=17, activation='sigmoid'))
  return loki

In [0]:
model=loki_model()

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
from keras.callbacks import TensorBoard

import tensorflow as tf
import datetime

!rm -rf ./logs/ 

In [0]:
import os

In [0]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(log_dir=logdir, histogram_freq=1)

In [0]:
#batch size = 256 , epochs = 3

In [25]:
%tensorboard --logdir fit

UsageError: Line magic function `%tensorboard` not found.


In [26]:
model_loki=model.fit(x_train, y_train,batch_size=256,epochs=3,verbose=1,validation_data=(x_valid, y_valid),callbacks=[tensorboard_callback])

Train on 30000 samples, validate on 10479 samples
Epoch 1/3
30000/30000 [==============================] - 45s 2ms/step - loss: 0.3109 - accuracy: 0.8854 - val_loss: 0.2269 - val_accuracy: 0.9110
Epoch 2/3
30000/30000 [==============================] - 46s 2ms/step - loss: 0.2349 - accuracy: 0.9100 - val_loss: 0.2099 - val_accuracy: 0.9170
Epoch 3/3
30000/30000 [==============================] - 46s 2ms/step - loss: 0.2183 - accuracy: 0.9149 - val_loss: 0.2041 - val_accuracy: 0.9183


In [0]:
# %tensorboard --logdir fit

#another model with extra dense layer

In [0]:
def loki_1():

  loki = keras.models.Sequential()

  loki.add(Conv2D(64, (3, 3), input_shape=(32, 32, 3),activation='relu'))
  loki.add(MaxPooling2D(pool_size=(2, 2)))

  loki.add(Conv2D(32, (3, 3), activation='relu'))
  loki.add(MaxPooling2D(pool_size=(2, 2)))

  loki.add(Flatten())

  loki.add(Dense(units=128, activation='relu'))
  loki.add(Dropout(0.5))

  loki.add(Dense(units=128, activation='relu'))
  loki.add(Dropout(0.3))

  loki.add(Dense(units=17, activation='sigmoid'))

  return loki

In [29]:
model=loki_1()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# %tensorboard --logdir logs/fit
loki_model = model.fit(x_train, y_train,batch_size=128,epochs=3,verbose=1,validation_data=(x_valid, y_valid),callbacks=[tensorboard_callback])

Train on 30000 samples, validate on 10479 samples
Epoch 1/3
30000/30000 [==============================] - 49s 2ms/step - loss: 0.2789 - accuracy: 0.8944 - val_loss: 0.2228 - val_accuracy: 0.9117
Epoch 2/3
30000/30000 [==============================] - 49s 2ms/step - loss: 0.2211 - accuracy: 0.9133 - val_loss: 0.2004 - val_accuracy: 0.9188
Epoch 3/3
30000/30000 [==============================] - 48s 2ms/step - loss: 0.2047 - accuracy: 0.9189 - val_loss: 0.1855 - val_accuracy: 0.9269


#different batch size = 64 , optimizer = sgd  and only 2 epochs

In [30]:
model=loki_1()
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
# %tensorboard --logdir logs/fit
loki_model = model.fit(x_train, y_train,batch_size=64,epochs=2,verbose=1,validation_data=(x_valid, y_valid),callbacks=[tensorboard_callback])

Train on 30000 samples, validate on 10479 samples
Epoch 1/2
30000/30000 [==============================] - 52s 2ms/step - loss: 0.4311 - accuracy: 0.8458 - val_loss: 0.2905 - val_accuracy: 0.9055
Epoch 2/2
30000/30000 [==============================] - 52s 2ms/step - loss: 0.3151 - accuracy: 0.8924 - val_loss: 0.2720 - val_accuracy: 0.9055


In [31]:
pip install -U tensorboard

Requirement already up-to-date: tensorboard in /usr/local/lib/python3.6/dist-packages (2.2.1)


In [0]:
# tensorboard dev upload --logdir logs/fit \
#     --name "loki experiment" \                               
#     --description "hyperparameters experiment"  


In [35]:
!tensorboard dev upload --logdir logs

2020-04-30 10:06:37.055222: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Data for the "graphs" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
Upload started and will continue reading any new data as it's added
to the logdir. To stop uploading, press Ctrl-C.
View your TensorBoard live at: https://tensorboard.dev/experiment/1KdwUyCbSxyNNBzn7N4GoA/
E0430 10:06:40.652624 140501771982720 uploader.py:770] Attempted to re-upload existing blob.  Skipping.
E0430 10:06:41.632715 140501771982720 uploader.py:770] Attempted to re-upload existing blob.  Skipping.

Upload stopped. View your TensorBoard at https://tensorboard.dev/experiment/1KdwUyCbSxyNNBzn7N4GoA/
^C


In [38]:
!tensorboard dev update-metadata --experiment_id '1KdwUyCbSxyNNBzn7N4GoA' --name "graded 3 model" --description "trying different hyperparameters"

2020-04-30 10:13:23.986466: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Data for the "graphs" plugin is now uploaded to TensorBoard.dev! Note that uploaded data is public. If you do not want to upload data for this plugin, use the "--plugins" command line argument.
